In [5]:
import os, re, math, json
import numpy as np
import pandas as pd
from IPython.display import display


csv_path = "uae_used_cars_10k (1).csv"


In [ ]:
# imports and dataset Path

In [ ]:
# loading data

In [6]:
df_raw = pd.read_csv(csv_path)
df_raw.info()
display(df_raw.head(5))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Make          10000 non-null  object
 1   Model         10000 non-null  object
 2   Year          10000 non-null  int64 
 3   Price         10000 non-null  int64 
 4   Mileage       10000 non-null  int64 
 5   Body Type     10000 non-null  object
 6   Cylinders     9895 non-null   object
 7   Transmission  10000 non-null  object
 8   Fuel Type     10000 non-null  object
 9   Color         10000 non-null  object
 10  Location      10000 non-null  object
 11  Description   10000 non-null  object
dtypes: int64(3), object(9)
memory usage: 937.6+ KB


,Make,Model,Year,Price,Mileage,Body Type,Cylinders,Transmission,Fuel Type,Color,Location,Description
0,toyota,camry,2016,47819,156500,Sedan,4,Automatic Transmission,Gasoline,Black,Dubai,"2016 toyota camry with Rear camera, Leather se..."
1,kia,sorento,2013,61250,169543,SUV,4,Automatic Transmission,Gasoline,Grey,Abu Dhabi,"2013 kia sorento with Sunroof, Adaptive cruise..."
2,mini,cooper,2023,31861,221583,Soft Top Convertible,4,Automatic Transmission,Gasoline,Grey,Dubai,"2023 mini cooper with Adaptive cruise control,..."
3,nissan,altima,2016,110322,69754,Sedan,4,Automatic Transmission,Gasoline,Red,Dubai,"2016 nissan altima with Rear camera, Adaptive ..."
4,toyota,land-cruiser-76-series,2020,139994,71399,Pick Up Truck,4,Manual Transmission,Gasoline,White,Dubai,2020 toyota land-cruiser-76-series with Adapti...


In [ ]:
# normalization, bucketeting slugs HELPER FUNCTION

In [7]:
def std_text(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip()
    x = re.sub(r"[\u2013\u2014]+", "-", x) 
    x = re.sub(r"\s+", " ", x)
    return x

def title_clean(x):
    if pd.isna(x) or not str(x).strip():
        return np.nan
    parts = re.split(r"(-|/)", str(x).lower())
    parts = [p.title() if p not in ["-","/"] else p for p in parts]
    return "".join(parts)

def map_by_dict(val, mapping):
    if pd.isna(val):
        return np.nan
    key = str(val).strip().lower()
    return mapping.get(key, val)

def emirate_normalize(x):
    if pd.isna(x): return np.nan
    key = str(x).strip().lower()
    mapping = {
        "dubai":"Dubai","dxb":"Dubai","دبي":"Dubai",
        "abu dhabi":"Abu Dhabi","auh":"Abu Dhabi","ابوظبي":"Abu Dhabi",
        "sharjah":"Sharjah","shj":"Sharjah",
        "ajman":"Ajman",
        "ras al khaimah":"Ras Al Khaimah","rak":"Ras Al Khaimah",
        "fujairah":"Fujairah",
        "umm al quwain":"Umm Al Quwain","uaq":"Umm Al Quwain",
        "al ain":"Al Ain","al-ain":"Al Ain",
    }
    return mapping.get(key, title_clean(x))

def model_normalize(model):
    if pd.isna(model): return np.nan
    m = std_text(model).lower()
    m = m.replace("_", "-")
    m = re.sub(r"\s*series\b", "", m) 
    m = re.sub(r"\s+", " ", m).strip()
    m = m.replace(" - ", "-")
    return title_clean(m)

def to_numeric_safe(x):
    try:
        if pd.isna(x) or str(x).strip()=="":
            return np.nan
        x = re.sub(r"[^\d.]", "", str(x))
        if x == "": return np.nan
        return float(x)
    except Exception:
        return np.nan

def iqr_bounds(s):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    return (q1 - 1.5 * iqr, q3 + 1.5 * iqr)

def round_to_nearest(x, base=5000):
    if pd.isna(x): return np.nan
    return int(base * round(float(x)/base))

def make_price_buckets(prices, n=4):
    # Quantile edges per Make, rounded to AED 5k
    if prices.dropna().empty:
        return [0, 50000, 100000, 150000, 200000]
    qs = sorted(set([0.0] + [i/n for i in range(1,n)] + [1.0]))
    edges = [round_to_nearest(prices.quantile(q), 5000) for q in qs]
    edges = [int(edges[0] or 0)] + [int(max(edges[i], edges[i-1] + 5000)) for i in range(1,len(edges))]
    return edges

def bucket_label(edges, v, currency="AED"):
    if pd.isna(v) or v < edges[0]:
        return f"Under {currency} {edges[0]:,}"
    for i in range(len(edges)-1):
        if edges[i] <= v < edges[i+1]:
            lo = edges[i]; hi = edges[i+1]
            return f"{currency} {lo:,}–{hi:,}"
    return f"{currency} {edges[-1]:,}+"

def slugify(x):
    if pd.isna(x): return ""
    s = re.sub(r"[^a-zA-Z0-9]+","-", str(x).strip().lower())
    s = re.sub(r"-+","-", s).strip("-")
    return s


In [8]:
df = df_raw.copy()

# Normalize text columns
for col in ["Make","Model","Body Type","Transmission","Fuel Type","Color","Location","Description"]:
    if col in df.columns:
        df[col] = df[col].apply(std_text)

# Title-case for Make; normalize Model variants
if "Make" in df.columns:
    df["Make"] = df["Make"].apply(title_clean)
if "Model" in df.columns:
    df["Model_raw"] = df["Model"]
    df["Model"] = df["Model"].apply(model_normalize)

# Controlled mappings
body_map = {
    "sedan":"Sedan","saloon":"Sedan","salon":"Sedan",
    "suv":"SUV","sport utility vehicle":"SUV","jeep":"SUV",
    "hatchback":"Hatchback","coupe":"Coupe","convertible":"Convertible",
    "pickup":"Pickup","pickup truck":"Pickup","truck":"Pickup",
    "mpv":"MPV","minivan":"MPV","van":"Van",
}
trans_map = {
    "auto":"Automatic","automatic":"Automatic","a/t":"Automatic","amt":"Automatic",
    "manual":"Manual","m/t":"Manual",
    "cvt":"CVT","dct":"DCT","dsg":"DCT",
}
fuel_map = {
    "petrol":"Petrol","gasoline":"Petrol","benzene":"Petrol","gas":"Petrol",
    "diesel":"Diesel","hybrid":"Hybrid","electric":"Electric","ev":"Electric",
}
color_map = {
    "white":"White","black":"Black","silver":"Silver","grey":"Grey","gray":"Grey",
    "blue":"Blue","red":"Red","green":"Green","gold":"Gold","beige":"Beige",
    "brown":"Brown","orange":"Orange","yellow":"Yellow","purple":"Purple",
}

if "Body Type" in df.columns:
    df["Body Type"] = df["Body Type"].apply(lambda v: map_by_dict(title_clean(v) if pd.notna(v) else v, {k:body_map[k] for k in body_map}))
if "Transmission" in df.columns:
    df["Transmission"] = df["Transmission"].apply(lambda v: map_by_dict(str(v).lower() if pd.notna(v) else v, trans_map))
if "Fuel Type" in df.columns:
    df["Fuel Type"] = df["Fuel Type"].apply(lambda v: map_by_dict(str(v).lower() if pd.notna(v) else v, fuel_map))
if "Color" in df.columns:
    df["Color"] = df["Color"].apply(lambda v: map_by_dict(str(v).lower() if pd.notna(v) else v, color_map))
if "Location" in df.columns:
    df["Location"] = df["Location"].apply(emirate_normalize)

# Numeric conversions
for col in ["Year","Price","Mileage","Cylinders"]:
    if col in df.columns:
        df[col] = df[col].apply(to_numeric_safe)

display(df.head(5))


,Make,Model,Year,Price,Mileage,Body Type,Cylinders,Transmission,Fuel Type,Color,Location,Description,Model_raw
0,Toyota,Camry,2016.0,47819.0,156500.0,Sedan,4.0,automatic transmission,Petrol,Black,Dubai,"2016 toyota camry with Rear camera, Leather se...",camry
1,Kia,Sorento,2013.0,61250.0,169543.0,SUV,4.0,automatic transmission,Petrol,Grey,Abu Dhabi,"2013 kia sorento with Sunroof, Adaptive cruise...",sorento
2,Mini,Cooper,2023.0,31861.0,221583.0,Soft Top Convertible,4.0,automatic transmission,Petrol,Grey,Dubai,"2023 mini cooper with Adaptive cruise control,...",cooper
3,Nissan,Altima,2016.0,110322.0,69754.0,Sedan,4.0,automatic transmission,Petrol,Red,Dubai,"2016 nissan altima with Rear camera, Adaptive ...",altima
4,Toyota,Land-Cruiser-76-,2020.0,139994.0,71399.0,Pick Up Truck,4.0,manual transmission,Petrol,White,Dubai,2020 toyota land-cruiser-76-series with Adapti...,land-cruiser-76-series


In [ ]:
# Removing duplicates and outliers

In [9]:
def approx_equal(a, b, tol=0.01):
    if pd.isna(a) and pd.isna(b): return True
    if pd.isna(a) or pd.isna(b): return False
    if a == 0: return a == b
    return abs(a-b) / max(abs(a), 1) <= tol

# Duplicate rule: identical car attributes; price & mileage within ±1%
if not df.empty:
    df["_dup_key"] = df[["Make","Model","Year","Body Type","Transmission","Fuel Type","Location"]].astype(str).agg("|".join, axis=1)
    df = df.sort_values(["_dup_key","Price","Mileage"]).copy()
    keep = []
    prev_key = None
    prev_price, prev_mileage = None, None
    for _, row in df.iterrows():
        key = row["_dup_key"]
        p = row.get("Price", np.nan)
        m = row.get("Mileage", np.nan)
        if key != prev_key or not (approx_equal(p, prev_price) and approx_equal(m, prev_mileage)):
            keep.append(True)
            prev_key, prev_price, prev_mileage = key, p, m
        else:
            keep.append(False)
    df = df[keep].drop(columns=["_dup_key"])

# Outliers by Make–Model using 1.5*IQR (applied only if group size >= 10)
if not df.empty and "Price" in df.columns and "Mileage" in df.columns:
    def filter_outliers(g):
        out = g.copy()
        if g["Price"].notna().sum() >= 10:
            lo, hi = iqr_bounds(g["Price"].dropna())
            out = out[(out["Price"].isna()) | ((out["Price"] >= lo) & (out["Price"] <= hi))]
        if out["Mileage"].notna().sum() >= 10:
            lo2, hi2 = iqr_bounds(out["Mileage"].dropna())
            out = out[(out["Mileage"].isna()) | ((out["Mileage"] >= lo2) & (out["Mileage"] <= hi2))]
        return out
    df = df.groupby(["Make","Model"], dropna=False, group_keys=False).apply(filter_outliers)

print("Rows after cleaning:", len(df))
display(df.head(5))


Rows after cleaning: 9698


C:\Users\Aarushi Daksh\AppData\Local\Temp\ipykernel_24288\1258255804.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(["Make","Model"], dropna=False, group_keys=False).apply(filter_outliers)


,Make,Model,Year,Price,Mileage,Body Type,Cylinders,Transmission,Fuel Type,Color,Location,Description,Model_raw
4573,Acura,Rdx,2020.0,89585.0,61814.0,SUV,4.0,automatic transmission,Petrol,Blue,Dubai,"2020 acura rdx with Bluetooth, Rear camera, Na...",rdx
982,Acura,Rdx,2024.0,124271.0,231490.0,SUV,4.0,automatic transmission,Petrol,Blue,Dubai,"2024 acura rdx with Sunroof, Rear camera. Cond...",rdx
387,Alfa-Romeo,156159,2007.0,93845.0,233123.0,Sedan,6.0,manual transmission,Petrol,Red,Dubai,"2007 alfa-romeo 156159 with Leather seats, Nav...",156159
3116,Alfa-Romeo,156159,2009.0,72592.0,259265.0,Sedan,6.0,manual transmission,Petrol,Red,Dubai,"2009 alfa-romeo 156159 with Leather seats, Nav...",156159
5075,Alfa-Romeo,4C,2021.0,170322.0,66553.0,Sports Car,4.0,automatic transmission,Petrol,Black,Dubai,"2021 alfa-romeo 4c with Sunroof, Navigation sy...",4c


In [ ]:
# summaries for Data_Audits

In [10]:
tot_by_make = df.groupby("Make", dropna=False, as_index=False).size().rename(columns={"size":"Total Listings"})
tot_by_loc  = df.groupby("Location", dropna=False, as_index=False).size().rename(columns={"size":"Total Listings"})
tot_by_body = df.groupby("Body Type", dropna=False, as_index=False).size().rename(columns={"size":"Total Listings"})

median_make_model = (
    df.groupby(["Make","Model"], dropna=False)
      .agg(Median_Price=("Price","median"), Median_Mileage=("Mileage","median"), Count=("Model","size"))
      .reset_index()
      .sort_values(["Make","Model"])
)

display(tot_by_make.head(10))
display(tot_by_loc.head(10))
display(tot_by_body.head(10))
display(median_make_model.head(10))


,Make,Total Listings
0,Acura,2
1,Alfa-Romeo,23
2,Aston-Martin,34
3,Audi,376
4,Baic,1
5,Bentley,125
6,Bmw,672
7,Cadillac,144
8,Changan,5
9,Chevrolet,296


,Location,Total Listings
0,Abu Dhabi,717
1,Ajman,172
2,Al Ain,46
3,Dubai,7738
4,Fujeirah,8
5,Ras Al Khaimah,18
6,Sharjah,989
7,Umm Al Qawain,10


,Body Type,Total Listings
0,Coupe,813
1,Crossover,132
2,Hard Top Convertible,136
3,Hatchback,325
4,Other,110
5,Pick Up Truck,420
6,SUV,4466
7,Sedan,2708
8,Soft Top Convertible,110
9,Sports Car,265


,Make,Model,Median_Price,Median_Mileage,Count
0,Acura,Rdx,106928.0,146652.0,2
1,Alfa-Romeo,156159,83218.5,246194.0,2
2,Alfa-Romeo,4C,170322.0,66553.0,1
3,Alfa-Romeo,Giulia,138112.0,179250.0,10
4,Alfa-Romeo,Giulietta,92827.0,197062.0,7
5,Alfa-Romeo,Stelvio,129817.0,51583.0,3
6,Aston-Martin,Db11,724657.0,192817.0,10
7,Aston-Martin,Db9,359404.0,76915.0,3
8,Aston-Martin,Dbs,1098470.5,221783.5,2
9,Aston-Martin,Other,973532.5,195569.5,2


In [ ]:
# Price buckets per make and lable each row 

In [11]:
price_edges_by_make = {}
if "Price" in df.columns and "Make" in df.columns and not df.empty:
    for mk, g in df.groupby("Make"):
        price_edges_by_make[mk] = make_price_buckets(g["Price"], n=4)

def label_price(row):
    mk = row.get("Make", np.nan)
    price = row.get("Price", np.nan)
    edges = price_edges_by_make.get(mk, [0,50000,100000,150000,200000])
    return bucket_label(edges, price, "AED")

if not df.empty:
    df["Price_Bucket"] = df.apply(label_price, axis=1)

display(df[["Make","Model","Price","Price_Bucket"]].head(10))


,Make,Model,Price,Price_Bucket
4573,Acura,Rdx,89585.0,"Under AED 90,000"
982,Acura,Rdx,124271.0,"AED 115,000–125,000"
387,Alfa-Romeo,156159,93845.0,"AED 50,000–95,000"
3116,Alfa-Romeo,156159,72592.0,"AED 50,000–95,000"
5075,Alfa-Romeo,4C,170322.0,"AED 140,000–245,000"
8759,Alfa-Romeo,Giulia,112769.0,"AED 95,000–115,000"
1122,Alfa-Romeo,Giulia,240463.0,"AED 140,000–245,000"
1237,Alfa-Romeo,Giulia,132408.0,"AED 115,000–140,000"
318,Alfa-Romeo,Giulia,126527.0,"AED 115,000–140,000"
2711,Alfa-Romeo,Giulia,132761.0,"AED 115,000–140,000"


In [ ]:
# 20 row pSEO blueprint

In [12]:
page_types = [
    ("Location + Make + Body Type", "/{location}/{make}/{body}/", "Used {Make} {Body} in {Location} | Best Deals", "Used {Make} {Body} in {Location}", "Browse {count}+ {Make} {Body} cars for sale in {Location}. Filter by price, mileage, year, and more."),
    ("Location + Make + Price Bucket", "/{location}/{make}/under-{price_slug}/", "Affordable {Make} cars in {Location} | {price_label}", "{Make} cars in {Location} | {price_label}", "Find {Make} cars in {Location} priced {price_label}. Fresh listings updated daily."),
    ("Make + Model + Year Range", "/{make}/{model}/{year_lo}-{year_hi}/", "Used {Make} {Model} {year_lo}–{year_hi} for Sale", "{Make} {Model} {year_lo}–{year_hi}", "Explore {Make} {Model} from {year_lo}–{year_hi}. See prices, mileage, and features."),
    ("Location + Body Type", "/{location}/{body}/", "Used {Body} cars in {Location}", "Used {Body} cars in {Location}", "Shop {Body} cars available today in {Location}. Compare prices and mileage."),
    ("Make + Model + Location", "/{make}/{model}/{location}/", "{Make} {Model} in {Location} | Listings & Prices", "{Make} {Model} for sale in {Location}", "See all {Make} {Model} listings in {Location} with median price and mileage insights."),
    ("Make + Price Bucket", "/{make}/under-{price_slug}/", "{Make} Cars {price_label}", "{Make} Cars {price_label}", "Budget-friendly {Make} listings priced {price_label}. Filter by year, mileage, and more."),
    ("Make + Body Type", "/{make}/{body}/", "{Make} {Body} for Sale", "{Make} {Body}", "Browse {Make} {Body} listings across the UAE with filters for transmission and fuel type."),
    ("Location + Make", "/{location}/{make}/", "{Make} cars in {Location}", "{Make} cars in {Location}", "All {Make} listings in {Location}. View prices, mileage, and verified sellers."),
    ("Make + Model + Price Bucket", "/{make}/{model}/under-{price_slug}/", "{Make} {Model} {price_label}", "{Make} {Model} {price_label}", "Find {Make} {Model} priced {price_label}. Check photos, specs, and service history notes."),
    ("Make + Model + Location + Price Bucket", "/{make}/{model}/{location}/under-{price_slug}/", "{Make} {Model} in {Location} | {price_label}", "{Make} {Model} in {Location} | {price_label}", "Tighten search for {Make} {Model} in {Location} priced {price_label}."),
    ("Location + Fuel Type", "/{location}/fuel/{fuel}/", "Used {Fuel} cars in {Location}", "{Fuel} cars in {Location}", "Explore {Fuel} listings in {Location}. Compare running costs and features."),
    ("Make + Transmission", "/{make}/gearbox/{trans}/", "{Make} {Trans} Cars", "{Make} {Trans}", "Shop {Make} cars with {Trans} transmission. Filter by year and mileage."),
    ("Make + Cylinders", "/{make}/cyl/{cyl}/", "{Make} {Cyl}-Cylinder Cars", "{Make} {Cyl}-Cylinder", "Browse {Cyl}-cylinder {Make} cars with power and efficiency insights."),
    ("Location + Make + Year Range", "/{location}/{make}/{year_lo}-{year_hi}/", "{Make} in {Location} {year_lo}–{year_hi}", "{Make} in {Location} {year_lo}–{year_hi}", "See {Make} stock in {Location} from {year_lo}–{year_hi}, with median pricing."),
    ("Body Type + Price Bucket", "/body/{body}/under-{price_slug}/", "{Body} cars {price_label}", "{Body} cars {price_label}", "Discover {Body} listings priced {price_label} across the UAE."),
    ("Make + Model + Body Type", "/{make}/{model}/{body}/", "{Make} {Model} {Body}", "{Make} {Model} {Body}", "Narrow to {Make} {Model} in {Body} style with features and owner notes."),
    ("Location + Color", "/{location}/color/{color}/", "{Color} cars in {Location}", "{Color} cars in {Location}", "Shop {Color} cars available today in {Location}."),
    ("Make + Model + Mileage Band", "/{make}/{model}/mileage/{mileage_band}/", "{Make} {Model} with {mileage_band} mileage", "{Make} {Model} | {mileage_band}", "Focus on {Make} {Model} with {mileage_band} mileage band to estimate wear."),
    ("Location + Make + Fuel", "/{location}/{make}/fuel/{fuel}/", "{Make} {Fuel} in {Location}", "{Make} {Fuel} in {Location}", "Fuel-specific {Make} stock in {Location} with running-cost proxies."),
    ("All Makes + Location", "/{location}/", "Used cars in {Location} | Latest Listings & Prices", "Used cars in {Location}", "All makes in {Location}. Use filters for body type, price, fuel, and more."),
]

def pick_top_or_fallback(series, fallback):
    vals = [v for v in series.dropna().astype(str).head(1).tolist()]
    return vals[0] if vals else fallback

# sample values from your data
sample_loc  = pick_top_or_fallback(tot_by_loc.sort_values("Total Listings", ascending=False)["Location"] if not tot_by_loc.empty else pd.Series(), "Dubai")
sample_make = pick_top_or_fallback(tot_by_make.sort_values("Total Listings", ascending=False)["Make"] if not tot_by_make.empty else pd.Series(), "Toyota")
sample_body = pick_top_or_fallback(tot_by_body.sort_values("Total Listings", ascending=False)["Body Type"] if not tot_by_body.empty else pd.Series(), "SUV")

example_edges = price_edges_by_make.get(sample_make, [0,50000,100000,150000,200000])
example_label = f"AED {example_edges[0]:,}–{example_edges[1]:,}" if len(example_edges) >= 2 else "Under AED 50,000"
price_slug = slugify(example_label.replace("AED","").strip())

top_model_row = median_make_model[median_make_model["Make"]==sample_make].sort_values("Count", ascending=False).head(1) if not median_make_model.empty else pd.DataFrame()
sample_model = top_model_row["Model"].iloc[0] if not top_model_row.empty else "Corolla"

year_lo = int(df["Year"].quantile(0.25)) if "Year" in df.columns and df["Year"].notna().any() else 2016
year_hi = int(df["Year"].quantile(0.75)) if "Year" in df.columns and df["Year"].notna().any() else 2022

sample_fuel = pick_top_or_fallback(df["Fuel Type"] if "Fuel Type" in df.columns else pd.Series(), "Petrol")
sample_trans = pick_top_or_fallback(df["Transmission"] if "Transmission" in df.columns else pd.Series(), "Automatic")
sample_color = pick_top_or_fallback(df["Color"] if "Color" in df.columns else pd.Series(), "White")
try:
    sample_cyl = int(float(pick_top_or_fallback(df["Cylinders"].dropna().astype(str) if "Cylinders" in df.columns else pd.Series(), 4)))
except:
    sample_cyl = 4

def mileage_bandize(m):
    if pd.isna(m): return "Under 50k km"
    if m < 50000: return "Under 50k km"
    if m < 100000: return "50k–100k km"
    if m < 150000: return "100k–150k km"
    return "150k+ km"
ex_mileage_band = "Under 50k km"

blue_rows = []
for pt_name, url_pat, title_pat, h1_pat, meta_pat in page_types:
    context = {
        "location": slugify(sample_loc),
        "Location": sample_loc,
        "make": slugify(sample_make),
        "Make": sample_make,
        "model": slugify(sample_model),
        "Model": sample_model,
        "body": slugify(sample_body),
        "Body": sample_body,
        "price_slug": price_slug,
        "price_label": example_label,
        "year_lo": year_lo,
        "year_hi": year_hi,
        "fuel": slugify(sample_fuel),
        "Fuel": sample_fuel,
        "trans": slugify(sample_trans),
        "Trans": sample_trans,
        "color": slugify(sample_color),
        "Color": sample_color,
        "cyl": sample_cyl,
        "Cyl": sample_cyl,
        "mileage_band": ex_mileage_band,
        "count": int(tot_by_make.loc[tot_by_make["Make"]==sample_make, "Total Listings"].sum()) if not tot_by_make.empty else 100
    }
    url = url_pat.format(**context)
    title = title_pat.format(**context)
    h1 = h1_pat.format(**context)
    meta = meta_pat.format(**context)
    reason = "High-intent combination that maps to common search patterns; scalable across dataset."
    volume_proxy = "Listings in segment (supply proxy)"
    blue_rows.append([pt_name, url, title, h1, meta, reason, volume_proxy])

blueprint_df = pd.DataFrame(blue_rows, columns=["Page Type","Example URL","Title","H1","Meta","Reason","Volume Proxy"])
display(blueprint_df.head(10))


,Page Type,Example URL,Title,H1,Meta,Reason,Volume Proxy
0,Location + Make + Body Type,/dubai/mercedes-benz/suv/,Used Mercedes-Benz SUV in Dubai | Best Deals,Used Mercedes-Benz SUV in Dubai,Browse 1445+ Mercedes-Benz SUV cars for sale i...,High-intent combination that maps to common se...,Listings in segment (supply proxy)
1,Location + Make + Price Bucket,/dubai/mercedes-benz/under-15-000-85-000/,Affordable Mercedes-Benz cars in Dubai | AED 1...,"Mercedes-Benz cars in Dubai | AED 15,000–85,000",Find Mercedes-Benz cars in Dubai priced AED 15...,High-intent combination that maps to common se...,Listings in segment (supply proxy)
2,Make + Model + Year Range,/mercedes-benz/s-class/2009-2019/,Used Mercedes-Benz S-Class 2009–2019 for Sale,Mercedes-Benz S-Class 2009–2019,Explore Mercedes-Benz S-Class from 2009–2019. ...,High-intent combination that maps to common se...,Listings in segment (supply proxy)
3,Location + Body Type,/dubai/suv/,Used SUV cars in Dubai,Used SUV cars in Dubai,Shop SUV cars available today in Dubai. Compar...,High-intent combination that maps to common se...,Listings in segment (supply proxy)
4,Make + Model + Location,/mercedes-benz/s-class/dubai/,Mercedes-Benz S-Class in Dubai | Listings & Pr...,Mercedes-Benz S-Class for sale in Dubai,See all Mercedes-Benz S-Class listings in Duba...,High-intent combination that maps to common se...,Listings in segment (supply proxy)
5,Make + Price Bucket,/mercedes-benz/under-15-000-85-000/,"Mercedes-Benz Cars AED 15,000–85,000","Mercedes-Benz Cars AED 15,000–85,000",Budget-friendly Mercedes-Benz listings priced ...,High-intent combination that maps to common se...,Listings in segment (supply proxy)
6,Make + Body Type,/mercedes-benz/suv/,Mercedes-Benz SUV for Sale,Mercedes-Benz SUV,Browse Mercedes-Benz SUV listings across the U...,High-intent combination that maps to common se...,Listings in segment (supply proxy)
7,Location + Make,/dubai/mercedes-benz/,Mercedes-Benz cars in Dubai,Mercedes-Benz cars in Dubai,All Mercedes-Benz listings in Dubai. View pric...,High-intent combination that maps to common se...,Listings in segment (supply proxy)
8,Make + Model + Price Bucket,/mercedes-benz/s-class/under-15-000-85-000/,"Mercedes-Benz S-Class AED 15,000–85,000","Mercedes-Benz S-Class AED 15,000–85,000","Find Mercedes-Benz S-Class priced AED 15,000–8...",High-intent combination that maps to common se...,Listings in segment (supply proxy)
9,Make + Model + Location + Price Bucket,/mercedes-benz/s-class/dubai/under-15-000-85-000/,"Mercedes-Benz S-Class in Dubai | AED 15,000–85...","Mercedes-Benz S-Class in Dubai | AED 15,000–85...",Tighten search for Mercedes-Benz S-Class in Du...,High-intent combination that maps to common se...,Listings in segment (supply proxy)


In [ ]:
# top 25

In [13]:
if not df.empty:
    seg = (df.groupby(["Make","Model"], dropna=False)
             .agg(
                 listings=("Model","size"),
                 median_price=("Price","median"),
                 loc_diversity=("Location", lambda s: s.dropna().nunique())
             )
             .reset_index())
else:
    seg = pd.DataFrame(columns=["Make","Model","listings","median_price","loc_diversity"])

if not seg.empty:
    seg["listings_n"] = (seg["listings"] - seg["listings"].min()) / (seg["listings"].max() - seg["listings"].min() + 1e-9)
    seg["loc_div_n"] = (seg["loc_diversity"] - seg["loc_diversity"].min()) / (seg["loc_diversity"].max() - seg["loc_diversity"].min() + 1e-9)
    inv_price = seg["median_price"].max() - seg["median_price"]
    seg["price_n"] = (inv_price - inv_price.min()) / (inv_price.max() - inv_price.min() + 1e-9)
    seg["priority_score"] = 0.5*seg["listings_n"] + 0.3*seg["loc_div_n"] + 0.2*seg["price_n"]
    seg_top25 = seg.sort_values("priority_score", ascending=False).head(25).copy()
    seg_top25["Why this first"] = (
        "High supply (more pages), good geographic coverage, and affordable medians → broader demand capture."
    )
else:
    seg_top25 = pd.DataFrame(columns=["Make","Model","listings","median_price","loc_diversity","priority_score","Why this first"])

display(seg_top25.head(10))


,Make,Model,listings,median_price,loc_diversity,listings_n,loc_div_n,price_n,priority_score,Why this first
395,Nissan,Patrol,375,133316.0,7,1.000000,1.000000,0.990922,0.998184,"High supply (more pages), good geographic cove..."
348,Mercedes-Benz,S-Class,255,178418.0,5,0.679144,0.666667,0.987651,0.737102,"High supply (more pages), good geographic cove..."
335,Mercedes-Benz,G-Class,254,910261.0,5,0.676471,0.666667,0.934582,0.725152,"High supply (more pages), good geographic cove..."
480,Toyota,Land-Cruiser,207,190618.0,6,0.550802,0.833333,0.986767,0.722754,"High supply (more pages), good geographic cove..."
269,Land-Rover,Range-Rover-Sport,191,163943.0,5,0.508021,0.666667,0.988701,0.651751,"High supply (more pages), good geographic cove..."
332,Mercedes-Benz,E-Class,188,93552.0,5,0.500000,0.666667,0.993806,0.648761,"High supply (more pages), good geographic cove..."
325,Mercedes-Benz,C-Class,188,119806.0,4,0.500000,0.500000,0.991902,0.598380,"High supply (more pages), good geographic cove..."
267,Land-Rover,Range-Rover,180,209091.5,4,0.478610,0.500000,0.985427,0.586390,"High supply (more pages), good geographic cove..."
426,Porsche,Cayenne,164,134960.0,4,0.435829,0.500000,0.990803,0.566075,"High supply (more pages), good geographic cove..."
470,Toyota,Camry,65,41540.0,6,0.171123,0.833333,0.997577,0.535077,"High supply (more pages), good geographic cove..."


In [ ]:
# Features extraction top 15 overall and per top 3 makes 

In [14]:
feature_keywords = [
    "rear camera","backup camera","reverse camera",
    "parking sensors","front sensors","rear sensors",
    "adaptive cruise control","cruise control",
    "leather seats","sunroof","panoramic roof","moonroof",
    "android auto","apple carplay","bluetooth",
    "navigation","gps",
    "keyless entry","push button start",
    "alloy wheels","fog lamps","led headlights",
    "abs","airbags","traction control","stability control",
    "automatic climate control","rear ac","dual zone",
]

def find_features(text):
    if pd.isna(text): return []
    t = str(text).lower()
    found = []
    for kw in feature_keywords:
        if kw in t:
            found.append(kw)
    return list(set(found))

if not df.empty and "Description" in df.columns:
    feats = df[["Make","Description"]].copy()
    feats["features"] = feats["Description"].apply(find_features)
    exploded = feats.explode("features")
    feat_counts = exploded["features"].value_counts().dropna().head(15).reset_index()
    feat_counts.columns = ["Feature","Frequency"]
    top3_makes = df["Make"].value_counts().dropna().head(3).index.tolist() if "Make" in df.columns else []
    per_make_tables = {}
    for mk in top3_makes:
        em = exploded[exploded["Make"]==mk]
        per_make = em["features"].value_counts().dropna().head(15).reset_index()
        per_make.columns = ["Feature",f"Frequency_{mk}"]
        per_make_tables[mk] = per_make
else:
    feat_counts = pd.DataFrame(columns=["Feature","Frequency"])
    per_make_tables = {}

display(feat_counts.head(10))
top3_makes


,Feature,Frequency
0,sunroof,4884
1,leather seats,4861
2,bluetooth,4855
3,navigation,4853
4,adaptive cruise control,4764
5,cruise control,4764
6,rear camera,4762


['Mercedes-Benz', 'Nissan', 'Toyota']

In [ ]:
# Excel and content block export 

In [17]:
out_xlsx = "seo_used_cars_assignment.xlsx"  # saved in current working dir

assumptions = [
    ["Normalization", "Trim/whitespace fix; dashes normalized; Title-case Make; normalized Model (remove 'series', unify hyphens)."],
    ["Mappings", "Body/Transmission/Fuel/Color mapped via dictionaries."],
    ["Location", "Mapped common abbreviations to UAE emirates (Dubai, Abu Dhabi, Sharjah, etc.)."],
    ["Duplicates", "Same Make/Model/Year/Body/Trans/Fuel/Location; Price & Mileage within ±1% => keep one."],
    ["Outliers", "Within Make–Model, dropped Price & Mileage outside 1.5×IQR (if >=10 samples)."],
    ["Price Buckets", "Per-Make quartiles rounded to nearest AED 5k; used for taxonomy & filters."],
    ["Features", "Keyword matching on Description; extendable list."],
    ["Volume Proxy", "Listings count used as supply proxy."],
    ["Indexation", "Index high-intent combos; canonicalize facet variants; noindex thin combinations."],
]
audit_tbl = pd.DataFrame(assumptions, columns=["Topic","Assumption / Rule"])

with pd.ExcelWriter(out_xlsx, engine="xlsxwriter") as writer:
    # Data_Audit: assumptions + three totals
    audit_tbl.to_excel(writer, index=False, sheet_name="Data_Audit")
    startrow = len(audit_tbl) + 2
    tot_by_make.to_excel(writer, index=False, sheet_name="Data_Audit", startrow=startrow)
    startrow += len(tot_by_make) + 3
    tot_by_loc.to_excel(writer, index=False, sheet_name="Data_Audit", startrow=startrow)
    startrow += len(tot_by_loc) + 3
    tot_by_body.to_excel(writer, index=False, sheet_name="Data_Audit", startrow=startrow)

    # Other sheets
    median_make_model.to_excel(writer, index=False, sheet_name="Median_By_MakeModel")
    blueprint_df.to_excel(writer, index=False, sheet_name="pSEO_Blueprint")
    seg_top25.to_excel(writer, index=False, sheet_name="Opportunities_Top25")
    feat_counts.to_excel(writer, index=False, sheet_name="Features_Overall")
    for mk, tbl in list(per_make_tables.items())[:3]:
        sheet = f"Features_{mk[:25]}"
        tbl.to_excel(writer, index=False, sheet_name=sheet)

print("Wrote:", out_xlsx)


Wrote: seo_used_cars_assignment.xlsx


In [16]:
!pip install xlsxwriter


In [ ]:
# generates 3 content blocks 

In [18]:
# Builds a small .md you can paste into your doc/PPT
make_for_blocks = pick_top_or_fallback(tot_by_make.sort_values("Total Listings", ascending=False)["Make"], "Toyota") if not tot_by_make.empty else "Toyota"
model_for_blocks = sample_model
loc_for_blocks = sample_loc
example_label_for_blocks = example_label

content_md = f"""# Modular Content Blocks

## Popular features for {make_for_blocks} in {loc_for_blocks}
- Based on listing descriptions, common features include: **{", ".join(feat_counts["Feature"].head(6).tolist()) if not feat_counts.empty else "rear camera, parking sensors, Android Auto, Apple CarPlay, sunroof, leather seats"}**.
- Use these as filter facets and highlight them in cards.

## What to check when buying a used {model_for_blocks} in UAE
- **Service history:** Prefer agency/service-center records; verify major services at 40k/80k/120k km.
- **Accident/repair checks:** Inspect panels for repainting; check underbody & suspension for desert/off-road wear.
- **Ownership costs:** Review **tyres**, **battery**, and **brake** life; cross-check insurance band for the model year.
- **Test drive:** Listen for transmission shifts and check AC performance at idle and highway speeds.

## Ownership cost proxies: Mileage and Year insights for {make_for_blocks}–{model_for_blocks}
- Median **mileage** and **price** help estimate depreciation curves.
- Target **Under 50k km** where possible; verify condition via photos and inspection notes.
- For budget pages ({example_label_for_blocks}), surface listings with full service history and lower owners count where available.
"""

with open("pSEO_Content_Blocks.md", "w", encoding="utf-8") as f:
    f.write(content_md)

print("Wrote: pSEO_Content_Blocks.md")


Wrote: pSEO_Content_Blocks.md
